<a href="https://colab.research.google.com/github/mr329/Design-Efficiency-V2/blob/main/Extract_fan_schedule_from_PDF_to_Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# FINAL WORKING SOLUTION: Extract fan schedule from PDF to Excel.
# This version uses character position analysis to properly identify columns.
# Modified for Google Colab compatibility with interactive file selection.


import sys
import subprocess

# Google Colab setup
try:
    from google.colab import drive
    from google.colab import files
    COLAB = True
except ImportError:
    COLAB = False

# Install required packages
print("Installing required packages...")
required_packages = ['pdfplumber', 'pandas', 'openpyxl']
for package in required_packages:
    try:
        __import__(package)
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])
        print(f"✓ {package} installed")

import pdfplumber
import pandas as pd
import re
from collections import defaultdict
from pathlib import Path

def extract_fan_data_from_page(page, page_num):
    """
    Extract fan data from a page using character position-based column detection.
    """
    text = page.extract_text()
    if not text:
        return None

    lines = text.split('\n')

    # Find key lines
    item_ref_idx = None
    make_idx = None

    for i, line in enumerate(lines):
        if 'ITEM REF' in line:
            item_ref_idx = i
        if line.startswith('Make') and 'Fantech' in line:
            make_idx = i
            break

    if item_ref_idx is None or make_idx is None:
        return None

    # Use the "Make" line to determine number of columns (count "Fantech" occurrences)
    make_line = lines[make_idx]
    num_fans = make_line.count('Fantech')

    print(f"      Found {num_fans} fans on page {page_num}")

    # Get fan ID lines
    id_line1 = lines[item_ref_idx + 1] if item_ref_idx + 1 < len(lines) else ""
    id_line2 = lines[item_ref_idx + 2] if item_ref_idx + 2 < len(lines) else ""

    # Build fan IDs from the two lines
    # Split both lines and take appropriate parts
    parts1 = id_line1.split()
    parts2 = id_line2.split()

    # Filter out non-ID parts from line 2
    parts2 = [p for p in parts2 if not any(kw in p.upper() for kw in
             ['SYSTEM', 'EXHAUST', 'AIR', 'SUPPLY', 'OUTSIDE', 'MAKE-UP', 'STAIR'])]

    # Strategy: line 1 might have prefixes (SVC-...), line 2 has suffixes
    # Match them up based on position

    # If line 1 has SVC prefixes, pair them with suffixes
    fan_ids = []
    prefix_parts = [p for p in parts1 if 'SVC-' in p or p.endswith('-')]

    if len(prefix_parts) > 0 and len(parts2) > 0:
        # Combine prefixes with suffixes
        for i in range(num_fans):
            prefix = prefix_parts[i] if i < len(prefix_parts) else ""
            suffix = parts2[i] if i < len(parts2) else ""
            fan_id = prefix + suffix
            fan_ids.append(fan_id)
    else:
        # Just use the parts we have
        all_parts = parts1 + parts2
        all_parts = [p for p in all_parts if not any(kw in p.upper() for kw in
                    ['SYSTEM', 'EXHAUST', 'AIR', 'SUPPLY'])]
        fan_ids = all_parts[:num_fans]

    # If we still don't have the right number, use generic names
    while len(fan_ids) < num_fans:
        fan_ids.append(f"Fan_{len(fan_ids)+1}")

    fan_ids = fan_ids[:num_fans]  # Trim to exact number

    print(f"      Fan IDs: {fan_ids}")

    # Now extract data for each parameter
    data = {fan_id: {} for fan_id in fan_ids}
    current_section = ""

    # Parameter definitions
    params = [
        'System', 'Area Served', 'Location', 'AG Coombs Drawing Number',
        'VBIS Tag', 'Make', 'Model No.', 'Type', 'Spare Capacity Reqd',
        'Selection Airflow Rate', 'Selection External Static Pressure',
        'Actual Airflow Rate', 'Actual Extenral Static Pressure',
        'Impeller Speed', 'Weight', 'Drive Type', 'Motor Type', 'Filter Type',
        'Motor', 'Absorbed Power', 'Voltage', 'Phase', 'FLA', 'Speed', 'Starter', 'MSSB',
        '63 Hz', '125 Hz', '250 Hz', '500 Hz', '1000 Hz', '2000 Hz', '4000 Hz', '8000 Hz', 'SPL @ 3m'
    ]

    unit_map = {
        'Selection Airflow Rate': '(L/s)',
        'Selection External Static Pressure': '(Pa)',
        'Actual Airflow Rate': '(L/s)',
        'Actual Extenral Static Pressure': '(Pa)',
        'Impeller Speed': '(RPM)',
        'Weight': '(kg)',
        'Motor': '(kW)',
        'Absorbed Power': '(kW)',
        'Voltage': '(V)',
        'FLA': '(Amps)',
        'Speed': '(Poles)',
        '63 Hz': '(dB)', '125 Hz': '(dB)', '250 Hz': '(dB)', '500 Hz': '(dB)',
        '1000 Hz': '(dB)', '2000 Hz': '(dB)', '4000 Hz': '(dB)', '8000 Hz': '(dB)',
        'SPL @ 3m': '(dBA)',
    }

    # Parse data lines
    for i in range(item_ref_idx + 3, len(lines)):
        line = lines[i]

        if not line.strip() or 'NOTES' in line:
            break

        # Check for section headers
        if line.strip() in ['ELECTRICAL', 'IN DUCT SOUND POWER LEVEL - INLET',
                           'IN DUCT SOUND POWER LEVEL - OUTLET']:
            current_section = line.strip() + " - "
            continue

        # Try to match parameter
        matched_param = None
        rest = line

        for p in params:
            if line.startswith(p):
                matched_param = p
                rest = line[len(p):].strip()
                break

        if not matched_param:
            continue

        # Build full parameter name
        full_param = current_section + matched_param
        if matched_param in unit_map:
            full_param += ' ' + unit_map[matched_param]

        # Remove units from values
        rest = re.sub(r'\([^)]+\)', '', rest).strip()

        # Split values - try single space first, then fall back to multiple spaces
        values = rest.split()

        # If we got too many values (like multi-word descriptions), try double space split
        if len(values) > num_fans + 2:
            values = re.split(r'\s{2,}', rest)
            values = [v.strip() for v in values if v.strip()]

        # Take only the number of values we need
        values = values[:num_fans]

        # Assign to fans
        for idx, fan_id in enumerate(fan_ids):
            if idx < len(values):
                data[fan_id][full_param] = values[idx]

    # Convert to DataFrame
    if not any(data.values()):
        return None

    df = pd.DataFrame(data)
    df.index.name = 'Parameter'

    return df, fan_ids


def get_pdf_file_path():
    """
    Get PDF file path from user (interactive for Colab, command line for local).
    """
    if COLAB:
        print("\n" + "=" * 80)
        print("GOOGLE COLAB MODE: FILE UPLOAD")
        print("=" * 80)
        print("\nPlease upload the PDF file when prompted...\n")

        uploaded_files = files.upload()

        if not uploaded_files:
            print("No file uploaded. Exiting.")
            sys.exit(1)

        # Get the first uploaded file
        pdf_filename = list(uploaded_files.keys())[0]
        # Correct the path to where Colab's files.upload() actually places files
        pdf_path = f"/content/{pdf_filename}"
        print(f"✓ File received: {pdf_filename}")
        return pdf_path
    else:
        # Local mode - ask for path
        print("\n" + "=" * 80)
        print("LOCAL MODE: PDF FILE PATH")
        print("=" * 80)
        pdf_path = input("\nEnter the full path to your PDF file: ").strip()

        if not Path(pdf_path).exists():
            print(f"Error: File not found at {pdf_path}")
            sys.exit(1)

        print(f"✓ File found: {pdf_path}")
        return pdf_path


def main():
    # Get PDF file path from user
    pdf_path = get_pdf_file_path()

    # Determine output path
    if COLAB:
        # Save to /tmp/ directory in Colab (will be deleted after session)
        output_excel = "/tmp/VicX_Fans_Final.xlsx"
        print(f"Output will be saved to: {output_excel}")
    else:
        # Local output - same directory as input
        input_path = Path(pdf_path)
        output_excel = str(input_path.parent / "VicX_Fans_Final.xlsx")
        print(f"Output will be saved to: {output_excel}")

    print("=" * 80)
    print("FAN SCHEDULE EXTRACTION - VicX Fans PDF")
    print("=" * 80)

    all_fans_data = []
    all_fan_ids_by_page = {}

    with pdfplumber.open(pdf_path) as pdf:
        print(f"\nProcessing {len(pdf.pages)} pages...\n")

        for page_num, page in enumerate(pdf.pages, 1):
            print(f"   Page {page_num}:")
            result = extract_fan_data_from_page(page, page_num)

            if result is not None:
                df_page, fan_ids = result
                all_fan_ids_by_page[page_num] = fan_ids

                # Rename columns to include page
                df_page.columns = [f"{col} (Page {page_num})" for col in df_page.columns]
                all_fans_data.append(df_page)
                print(f"      ✓ Extracted {len(fan_ids)} fans with {len(df_page)} parameters")
            else:
                print(f"      ✗ No data extracted")

    if not all_fans_data:
        print("\n✗ No data extracted from PDF")
        return

    # Combine all pages horizontally (stack columns)
    print(f"\nCombining {len(all_fans_data)} pages...")
    df_stacked = pd.concat(all_fans_data, axis=1)

    print("\n" + "=" * 80)
    print("EXTRACTION SUMMARY")
    print("=" * 80)
    print(f"Total fans extracted: {len(df_stacked.columns)}")
    print(f"Total parameters: {len(df_stacked)}")

    # Create long format
    df_long = df_stacked.reset_index().melt(
        id_vars='Parameter',
        var_name='Fan_ID',
        value_name='Value'
    )

    # Extract page number
    df_long['Page'] = df_long['Fan_ID'].str.extract(r'\(Page (\d+)\)')[0]
    df_long['Fan_ID'] = df_long['Fan_ID'].str.replace(r' \(Page \d+\)', '', regex=True)
    df_long = df_long[df_long['Value'].notna() & (df_long['Value'] != '')]

    # Save to Excel
    print("\n" + "=" * 80)
    print("SAVING TO EXCEL")
    print("=" * 80)

    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        # Main output: Stacked view (parameters as rows, fans as columns)
        df_stacked.to_excel(writer, sheet_name='Stacked View')
        print(f"✓ 'Stacked View': {df_stacked.shape}")
        print(f"  Parameters (rows) × Fans (columns)")
        print(f"  👉 THIS IS THE HORIZONTALLY STACKED FORMAT")

        # Transposed view (fans as rows, parameters as columns)
        df_transposed = df_stacked.T
        df_transposed.to_excel(writer, sheet_name='Fans as Rows')
        print(f"✓ 'Fans as Rows': {df_transposed.shape}")
        print(f"  Fans (rows) × Parameters (columns)")

        # Long format
        df_long.to_excel(writer, sheet_name='Long Format', index=False)
        print(f"✓ 'Long Format': {df_long.shape}")
        print(f"  One row per fan-parameter combination")

    print("\n" + "=" * 80)
    print("✓ EXTRACTION COMPLETE!")
    print(f"  Output file: {output_excel}")
    print("  To download your Excel file:\n")
    print("  1. Click the 'Files' icon (folder) on the left sidebar.\n")
    print("  2. Navigate to the '/tmp/' directory.\n")
    print("  3. Right-click on 'VicX_Fans_Final.xlsx' and select 'Download'.\n")
    print("  (Note: Files in /tmp/ are temporary and will be deleted when your session ends.)")
    print("=" * 80)

    # # Show sample (COMMENTED OUT)
    # print("\nSample of Stacked View (first 20 parameters, first 10 fans):")
    # sample_cols = min(10, len(df_stacked.columns))
    # print(df_stacked.iloc[:20, :sample_cols].to_string())

    # List all fans in a table format
    print("\n" + "=" * 80)
    print(f"ALL {len(df_stacked.columns)} FANS EXTRACTED (Summary by Page):")
    print("=" * 80)

    fan_summary_data = []
    for page_num, fan_ids in sorted(all_fan_ids_by_page.items()):
        fan_summary_data.append({'Page': page_num, 'Number of Fans': len(fan_ids), 'Fan IDs': ', '.join(fan_ids)})

    df_fan_summary = pd.DataFrame(fan_summary_data)
    # Set display options to avoid truncation for 'Fan IDs' column
    pd.set_option('display.max_colwidth', None)
    print(df_fan_summary.to_string(index=False))


if __name__ == "__main__":
    main()

Installing required packages...
✓ pdfplumber already installed
✓ pandas already installed
✓ openpyxl already installed

GOOGLE COLAB MODE: FILE UPLOAD

Please upload the PDF file when prompted...



Saving VicX_Fans (1).pdf to VicX_Fans (1) (6).pdf
✓ File received: VicX_Fans (1) (6).pdf
Output will be saved to: /tmp/VicX_Fans_Final.xlsx
FAN SCHEDULE EXTRACTION - VicX Fans PDF

Processing 15 pages...

   Page 1:
      Found 5 fans on page 1
      Fan IDs: ['EF-01', 'CSF-03', 'SF-01-1', 'SF-01-2', 'EF-01-1']
      ✓ Extracted 5 fans with 41 parameters
   Page 2:
      Found 5 fans on page 2
      Fan IDs: ['EF-01-2', 'EF-02', 'CSF-04', 'SF-02', 'EF-01']
      ✓ Extracted 5 fans with 42 parameters
   Page 3:
      Found 5 fans on page 3
      Fan IDs: ['EF-02', 'EF-03', 'EF-04-1', 'EF-04-2', 'EF-05']
      ✓ Extracted 5 fans with 40 parameters
   Page 4:
      Found 5 fans on page 4
      Fan IDs: ['CSF-07', 'EF-08', 'SF-01-1', 'SF-01-2', 'EF-01-1']
      ✓ Extracted 5 fans with 41 parameters
   Page 5:
      Found 5 fans on page 5
      Fan IDs: ['EF-01-2', 'EF-02', 'EF-03', 'EF-06-1', 'EF-06-2']
      ✓ Extracted 5 fans with 39 parameters
   Page 6:
      Found 5 fans on page 6
   